Парсер описаний видео и комментариев с youtube.<br>Позволяет собрать данные с определенного канала за необходимый временной период.<br>Перечень видео с канала и текстовое описание каждого видео собираются через YouTube Data API.<br>Для скачивания комментариев используется скрипт youtube-comment-downloader, работающий не через API:<br>https://github.com/egbertbouman/youtube-comment-downloader

In [204]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import os
import glob
import re

In [205]:
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Set DEVELOPER_KEY to the API key value from the APIs & auth > Registered apps
# tab of https://cloud.google.com/console
# Please ensure that you have enabled the YouTube Data API for your project.
DEVELOPER_KEY = '' # enter your key here
YOUTUBE_API_SERVICE_NAME = 'youtube'
YOUTUBE_API_VERSION = 'v3'

def get_channelId(channelName):
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY)
    
    search_response = youtube.channels().list(    
    part='snippet',
    forUsername=channelName
    ).execute()
    channelId = search_response['items'][0]['id']
    return channelId

def get_videos_list(channelName, publishedAfter, publishedBefore, maxresults):
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY)

    channelId = get_channelId(channelName)
    
    search_response = youtube.search().list(    
    part='snippet',
    publishedAfter=publishedAfter,
    publishedBefore=publishedBefore,    
    maxResults=maxresults,
    channelId=channelId,
    type=''
    ).execute()
    
    videos_list = {'ID': [], 'Title': [], 'Lead sentences': [], 'Publish Date': []}
    for search_result in search_response.get('items', []):
        if search_result['id']['kind'] == 'youtube#video':
            videos_list['ID'].append(search_result['id']['videoId'])
            videos_list['Title'].append(search_result['snippet']['title'].replace('\r', '').replace('\n', '').replace('|', ''))
            publishDate = datetime.strptime(search_result['snippet']['publishedAt'][:10], '%Y-%m-%d')
            videos_list['Publish Date'].append(publishDate)
            videos_list['Lead sentences'].append(get_video_description(search_result['id']['videoId']))
    return videos_list

def get_video_description(videoId):
    youtube = build(YOUTUBE_API_SERVICE_NAME, YOUTUBE_API_VERSION,
    developerKey=DEVELOPER_KEY)
    
    search_response = youtube.videos().list(    
    id=videoId,
    part='snippet'
    ).execute()

    return search_response['items'][0]['snippet']['description'].replace('\r', '').replace('\n', '').replace('|', '')

In [206]:
def get_youtube_articles(channelName, startDate, endDate):
    startDate = datetime.strptime(startDate, '%d.%m.%Y')
    endDate = datetime.strptime(endDate, '%d.%m.%Y')
    date = startDate
    articles_list = {'ID': [], 'Title': [], 'Lead sentences': [], 'Publish Date': []}
    while date <= endDate:
        startDateRFC3339 = date.isoformat('T')[:20] + '.000Z'
        endDateRFC3339 = (date + timedelta(days=1)).isoformat('T')[:20] + '.000Z'
        day_articles = get_videos_list(channelName, startDateRFC3339, endDateRFC3339, 50)
        for key in articles_list.keys():
            articles_list[key].extend(day_articles[key])
        date += timedelta(days=1)
    return articles_list

In [207]:
def youtube_scrapper(channelName, startDate, endDate):
    # скраплю краткую информацию о видео, пишу в csv:
    posts = get_youtube_articles(channelName, startDate, endDate)
    print("Собрана информация о постах c ", startDate, " по ", endDate, {key: len(value) for key, value in posts.items()})
    df = pd.DataFrame(data = posts)
    df.to_csv('./csv/youtube_' + channelName + '_' + str(startDate) +'_posts.csv', sep='|', index=False, encoding='utf-8')
    # скраплю комментарии, пишу в csv:
    i = 1
    list_of_df = []
    for postId in posts['ID']:
        output = './csv/json/youtube_' + channelName + '_' + postId + '.json'
        # для скачивания комментариев использую youtube-comment-downloader, работающий не через API
        # https://github.com/egbertbouman/youtube-comment-downloader
        command = './youtube-comment-downloader-master/downloader.py --youtubeid ' + postId + ' --output ' + output
        #TODO поправить код downloader.py, чтобы нормально обрабатывались youtubeid, начинающиеся с "-"  
        %run -i $command
        try:
            df = pd.read_json(output, encoding='utf-8', lines=True)
            df['URL'] = [('https://www.youtube.com/watch?v='+ postId) for x in df['cid']]
            df.drop(['cid'], axis=1, inplace=True)
            list_of_df.append(df)
            print('Successful, {:.2f}% completed'.format(i*100/len(posts['ID'])))
            i += 1
        except:
            print('Unsuccessful for {}, {:.2f}% completed'.format(postId, i*100/len(posts['ID'])))
            i += 1
            pass

    df_comments = pd.concat(list_of_df)
    df_comments.dropna(inplace=True)
    df_comments.drop(['author', 'time'], axis=1, inplace=True)
    df_comments['Resource'] = 'youtube_' + channelName
    df_comments.rename(index=str, columns={"text": "comment"}, inplace=True)
    df_comments.to_csv('./csv/youtube_' + channelName + '_' + str(startDate) +'_comments.csv', sep='|', index=False, encoding='utf-8')

In [197]:
youtube_scrapper('tvrainru', '01.01.2018', '12.03.2018')

Собрана информация о постах c  01.01.2018  по  12.03.2018 {'ID': 109, 'Title': 109, 'Lead Sentences': 109, 'Publish Date': 109}
Downloaded 265 comment(s)
Done!
Successful, 0.92% completed
Downloaded 151 comment(s)
Done!
Successful, 1.83% completed
Downloaded 13 comment(s)
Done!
Successful, 2.75% completed
Downloaded 287 comment(s)
Done!
Successful, 3.67% completed
Downloaded 500 comment(s)
Done!
Successful, 4.59% completed
Downloaded 260 comment(s)
Done!
Successful, 5.50% completed
Downloaded 20 comment(s)
Done!
Successful, 6.42% completed

Done!
Unsuccessful, 7.34% completed
Downloaded 739 comment(s)
Done!
Successful, 8.26% completed
Downloaded 88 comment(s)
Done!
Successful, 9.17% completed
Downloaded 186 comment(s)
Done!
Successful, 10.09% completed
Downloaded 122 comment(s)
Done!
Successful, 11.01% completed
Downloaded 145 comment(s)
Done!
Successful, 11.93% completed
Downloaded 255 comment(s)
Done!
Successful, 12.84% completed
Downloaded 300 comment(s)
Done!
Successful, 13.76% com

Downloaded 51 comment(s)
Done!
Successful, 67.89% completed
Downloaded 349 comment(s)
Done!
Successful, 68.81% completed
Downloaded 539 comment(s)
Done!
Successful, 69.72% completed
Downloaded 611 comment(s)
Done!
Successful, 70.64% completed
Downloaded 1027 comment(s)
Done!
Successful, 71.56% completed
Downloaded 447 comment(s)
Done!
Successful, 72.48% completed
Downloaded 202 comment(s)
Done!
Successful, 73.39% completed
Downloaded 493 comment(s)
Done!
Successful, 74.31% completed
Downloaded 106 comment(s)
Done!
Successful, 75.23% completed
Downloaded 340 comment(s)
Done!
Successful, 76.15% completed
Downloaded 803 comment(s)
Done!
Successful, 77.06% completed
Downloaded 435 comment(s)
Done!
Successful, 77.98% completed
Downloaded 521 comment(s)
Done!
Successful, 78.90% completed
Downloaded 70 comment(s)
Done!
Successful, 79.82% completed
Downloaded 149 comment(s)
Done!
Successful, 80.73% completed
Downloaded 418 comment(s)
Done!
Successful, 81.65% completed
Downloaded 34 comment(s)


In [198]:
%%time
youtube_scrapper('tvrainru', '01.01.2017', '31.12.2017')

Собрана информация о постах c  01.01.2017  по  31.12.2017 {'ID': 440, 'Title': 440, 'Lead Sentences': 440, 'Publish Date': 440}
Downloaded 90 comment(s)
Done!
Successful, 0.23% completed
Downloaded 124 comment(s)
Done!
Successful, 0.45% completed
Downloaded 235 comment(s)
Done!
Successful, 0.68% completed
Downloaded 38 comment(s)
Done!
Successful, 0.91% completed
Downloaded 124 comment(s)
Done!
Successful, 1.14% completed
Downloaded 572 comment(s)
Done!
Successful, 1.36% completed
Downloaded 23 comment(s)
Done!
Successful, 1.59% completed
Downloaded 2232 comment(s)
Done!
Successful, 1.82% completed
Downloaded 211 comment(s)
Done!
Successful, 2.05% completed
Downloaded 176 comment(s)
Done!
Successful, 2.27% completed
Downloaded 567 comment(s)
Done!
Successful, 2.50% completed
Downloaded 211 comment(s)
Done!
Successful, 2.73% completed
Downloaded 107 comment(s)
Done!
Successful, 2.95% completed
Downloaded 37 comment(s)
Done!
Successful, 3.18% completed
Downloaded 12 comment(s)
Done!
Succ

usage: downloader.py [--help] [--youtubeid YOUTUBEID] [--output OUTPUT]
                     [--limit LIMIT]
downloader.py: error: argument --youtubeid/-y: expected one argument


SystemExit: 2

Unsuccessful, 4.32% completed
Downloaded 51 comment(s)
Done!
Successful, 4.55% completed
Downloaded 37 comment(s)
Done!
Successful, 4.77% completed
Downloaded 48 comment(s)
Done!
Successful, 5.00% completed
Downloaded 17 comment(s)
Done!
Successful, 5.23% completed
Downloaded 42 comment(s)
Done!
Successful, 5.45% completed
Downloaded 32 comment(s)
Done!
Successful, 5.68% completed
Downloaded 46 comment(s)
Done!
Successful, 5.91% completed
Downloaded 169 comment(s)
Done!
Successful, 6.14% completed
Downloaded 306 comment(s)
Done!
Successful, 6.36% completed
Downloaded 14 comment(s)
Done!
Successful, 6.59% completed
Downloaded 31 comment(s)
Done!
Successful, 6.82% completed
Downloaded 19 comment(s)
Done!
Successful, 7.05% completed
Downloaded 104 comment(s)
Done!
Successful, 7.27% completed
Downloaded 41 comment(s)
Done!
Successful, 7.50% completed
Downloaded 148 comment(s)
Done!
Successful, 7.73% completed
Downloaded 206 comment(s)
Done!
Successful, 7.95% completed
Downloaded 249 commen

usage: downloader.py [--help] [--youtubeid YOUTUBEID] [--output OUTPUT]
                     [--limit LIMIT]
downloader.py: error: argument --youtubeid/-y: expected one argument


...

usage: downloader.py [--help] [--youtubeid YOUTUBEID] [--output OUTPUT]
                     [--limit LIMIT]
downloader.py: error: argument --youtubeid/-y: expected one argument


SystemExit: 2

Unsuccessful, 82.27% completed
Downloaded 113 comment(s)
Done!
Successful, 82.50% completed
Downloaded 133 comment(s)
Done!
Successful, 82.73% completed
Downloaded 68 comment(s)
Done!
Successful, 82.95% completed
Downloaded 3413 comment(s)
Done!
Successful, 83.18% completed
Downloaded 425 comment(s)
Done!
Successful, 83.41% completed
Downloaded 1757 comment(s)
Done!
Successful, 83.64% completed
Downloaded 80 comment(s)
Done!
Successful, 83.86% completed
Downloaded 13 comment(s)
Done!
Successful, 84.09% completed
Downloaded 4 comment(s)
Done!
Successful, 84.32% completed
Downloaded 871 comment(s)
Done!
Successful, 84.55% completed
Downloaded 45 comment(s)
Done!
Successful, 84.77% completed

Done!
Unsuccessful, 85.00% completed
Downloaded 110 comment(s)
Done!
Successful, 85.23% completed
Downloaded 657 comment(s)
Done!
Successful, 85.45% completed
Downloaded 89 comment(s)
Done!
Successful, 85.68% completed
Downloaded 64 comment(s)
Done!
Successful, 85.91% completed
Downloaded 81 comment(

SystemExit: 1

Successful, 87.27% completed
Downloaded 54 comment(s)
Done!
Successful, 87.50% completed
Downloaded 2721 comment(s)
Done!
Successful, 87.73% completed
Downloaded 170 comment(s)
Done!
Successful, 87.95% completed
Downloaded 214 comment(s)
Done!
Successful, 88.18% completed
Downloaded 39 comment(s)
Done!
Successful, 88.41% completed
Downloaded 5813 comment(s)
Done!
Successful, 88.64% completed
Downloaded 93 comment(s)
Done!
Successful, 88.86% completed
Downloaded 47 comment(s)
Done!
Successful, 89.09% completed

Done!
Unsuccessful, 89.32% completed
Downloaded 23 comment(s)
Done!
Successful, 89.55% completed
Downloaded 1332 comment(s)
Done!
Successful, 89.77% completed
Downloaded 200 comment(s)
Done!
Successful, 90.00% completed
Downloaded 333 comment(s)
Done!
Successful, 90.23% completed
Downloaded 48 comment(s)
Done!
Successful, 90.45% completed
Downloaded 204 comment(s)
Done!
Successful, 90.68% completed
Downloaded 613 comment(s)
Done!
Successful, 90.91% completed
Downloaded 86 comment

In [208]:
youtube_scrapper('russia24tv', '01.01.2018', '12.03.2018')

Собрана информация о постах c  01.01.2018  по  12.03.2018 {'ID': 3042, 'Title': 3042, 'Lead sentences': 3042, 'Publish Date': 3042}
Downloaded 252 comment(s)
Done!
Successful, 0.03% completed
Downloaded 222 comment(s)
Done!
Successful, 0.07% completed
Downloaded 75 comment(s)
Done!
Successful, 0.10% completed
Downloaded 213 comment(s)
Done!
Successful, 0.13% completed
Downloaded 40 comment(s)
Done!
Successful, 0.16% completed
Downloaded 35 comment(s)
Done!
Successful, 0.20% completed
Downloaded 29 comment(s)
Done!
Successful, 0.23% completed
Downloaded 256 comment(s)
Done!
Successful, 0.26% completed
Downloaded 603 comment(s)
Done!
Successful, 0.30% completed
Downloaded 42 comment(s)
Done!
Successful, 0.33% completed
Downloaded 174 comment(s)
Done!
Successful, 0.36% completed
Downloaded 14 comment(s)
Done!
Successful, 0.39% completed
Downloaded 63 comment(s)
Done!
Successful, 0.43% completed
Downloaded 345 comment(s)
Done!
Successful, 0.46% completed
Downloaded 88 comment(s)
Done!
Succ

usage: downloader.py [--help] [--youtubeid YOUTUBEID] [--output OUTPUT]
                     [--limit LIMIT]
downloader.py: error: argument --youtubeid/-y: expected one argument


SystemExit: 2

Unsuccessful for -NW9hH05w28, 4.04% completed
Downloaded 110 comment(s)
Done!
Successful, 4.08% completed
Downloaded 207 comment(s)
Done!
Successful, 4.11% completed
Downloaded 210 comment(s)
Done!
Successful, 4.14% completed
Downloaded 447 comment(s)
Done!
Successful, 4.17% completed
Downloaded 127 comment(s)
Done!
Successful, 4.21% completed
Downloaded 76 comment(s)
Done!
Successful, 4.24% completed
Downloaded 120 comment(s)
Done!
Successful, 4.27% completed
Downloaded 38 comment(s)
Done!
Successful, 4.31% completed
Downloaded 76 comment(s)
Done!
Successful, 4.34% completed
Downloaded 70 comment(s)
Done!
Successful, 4.37% completed
Downloaded 235 comment(s)
Done!
Successful, 4.40% completed
Downloaded 28 comment(s)
Done!
Successful, 4.44% completed
Downloaded 195 comment(s)
Done!
Successful, 4.47% completed
Downloaded 16 comment(s)
Done!
Successful, 4.50% completed
Downloaded 9 comment(s)
Done!
Successful, 4.54% completed
Downloaded 86 comment(s)
Done!
Successful, 4.57% completed
Dow

Downloaded 16 comment(s)
Done!
Successful, 6.48% completed
Downloaded 284 comment(s)
Done!
Successful, 6.51% completed
Downloaded 55 comment(s)
Done!
Successful, 6.54% completed
Downloaded 3 comment(s)
Done!
Successful, 6.57% completed
Downloaded 381 comment(s)
Done!
Successful, 6.61% completed
Downloaded 33 comment(s)
Done!
Successful, 6.64% completed
Downloaded 156 comment(s)
Done!
Successful, 6.67% completed
Downloaded 1777 comment(s)
Done!
Successful, 6.71% completed
Downloaded 108 comment(s)
Done!
Successful, 6.74% completed
Downloaded 6 comment(s)
Done!
Successful, 6.77% completed
Downloaded 51 comment(s)
Done!
Successful, 6.80% completed
Downloaded 2 comment(s)
Done!
Successful, 6.84% completed

Done!
Unsuccessful for 7zIMyMehxSs, 6.87% completed
Downloaded 2 comment(s)
Done!
Successful, 6.90% completed

Done!
Unsuccessful for gOYjzUSOySQ, 6.94% completed
Downloaded 113 comment(s)
Done!
Successful, 6.97% completed
Downloaded 61 comment(s)
Done!
Successful, 7.00% completed
Downlo

...

SystemExit: 2

Unsuccessful for -hOu1toQqpo, 96.29% completed
Downloaded 77 comment(s)
Done!
Successful, 96.32% completed
Downloaded 55 comment(s)
Done!
Successful, 96.35% completed
Downloaded 45 comment(s)
Done!
Successful, 96.38% completed
Downloaded 24 comment(s)
Done!
Successful, 96.42% completed
Downloaded 4 comment(s)
Done!
Successful, 96.45% completed
Downloaded 16 comment(s)
Done!
Successful, 96.48% completed
Downloaded 44 comment(s)
Done!
Successful, 96.52% completed
Downloaded 39 comment(s)
Done!
Successful, 96.55% completed
Downloaded 78 comment(s)
Done!
Successful, 96.58% completed
Downloaded 120 comment(s)
Done!
Successful, 96.61% completed
Downloaded 56 comment(s)
Done!
Successful, 96.65% completed
Downloaded 15 comment(s)
Done!
Successful, 96.68% completed
Downloaded 1 comment(s)
Done!
Successful, 96.71% completed
Downloaded 156 comment(s)
Done!
Successful, 96.75% completed
Downloaded 66 comment(s)
Done!
Successful, 96.78% completed
Downloaded 141 comment(s)
Done!
Successful, 96.81% co

usage: downloader.py [--help] [--youtubeid YOUTUBEID] [--output OUTPUT]
                     [--limit LIMIT]
downloader.py: error: argument --youtubeid/-y: expected one argument


SystemExit: 2

Unsuccessful for -5Lwm34c3VQ, 98.06% completed

Done!
Unsuccessful for usF2Nug35ug, 98.09% completed
Downloaded 9 comment(s)
Done!
Successful, 98.13% completed
Downloaded 8 comment(s)
Done!
Successful, 98.16% completed
Downloaded 148 comment(s)
Done!
Successful, 98.19% completed
Downloaded 622 comment(s)
Done!
Successful, 98.22% completed
Downloaded 18 comment(s)
Done!
Successful, 98.26% completed
Downloaded 158 comment(s)
Done!
Successful, 98.29% completed
Downloaded 24 comment(s)
Done!
Successful, 98.32% completed
Downloaded 16 comment(s)
Done!
Successful, 98.36% completed
Downloaded 261 comment(s)
Done!
Successful, 98.39% completed
Downloaded 32 comment(s)
Done!
Successful, 98.42% completed
Downloaded 5 comment(s)
Done!
Successful, 98.45% completed
Downloaded 34 comment(s)
Done!
Successful, 98.49% completed
Downloaded 7 comment(s)
Done!
Successful, 98.52% completed
Downloaded 284 comment(s)
Done!
Successful, 98.55% completed
Downloaded 21 comment(s)
Done!
Successful, 98.59% complete